# Obtaining email addresses from urls using python
## Author: Alexandra Joukova
    

This code performs a Google search based on an input query and returns N urls based on the search (N defined by user). The code then loops through the urls and any urls linked to them and collects email addresses. The output is a unique list of emails.

In [4]:
import logging

import re
from requests_html import HTMLSession
import requests
import time
import requests.adapters
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("urllib3").setLevel(logging.DEBUG)

# scrape emails
from urllib.parse import urlsplit
from collections import deque
import collections, itertools
from bs4 import BeautifulSoup
import sys

import nest_asyncio
nest_asyncio.apply()

In [6]:
def get_source(url):
    try:
        session = HTMLSession()
        retry = Retry(connect=0, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        #print("start")
        
        headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
        "accept-encoding": "gzip, deflate, br",
        "cache-control": "max-age=0",
        "content-type": "application/x-www-form-urlencoded",
        "dnt": "1",
        "origin": "https://aavtrain.com",
        "referer": "https://aavtrain.com/index.asp",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) snap Chromium/74.0.3729.169 Chrome/74.0.3729.169 Safari/537.36"
        }
        
        response = session.get(url, verify=False, timeout=3.5, headers=headers)
        # response = session.get(url, verify=False, timeout=(0.5, 0.5))
        #print("done")
        return response
    except BaseException as e:
        print("EXCEPTION: ---------------------------")
        print(e)
        #print("--------------------------------------")
        return None

In [5]:
def scrap_emails(urls, max_links):
    visited_num_times = {}
    visited_num_emails = {}
    
    emails_list = []
    for original_url in urls:
        # read url from input
        # original_url = urls[0]
        # to save urls to be scraped
        unscraped = deque([original_url])
        
        # to save scraped urls
        scraped = set()
        
        # to save fetched emails
        emails = set() 

        len_unscraped = 0
        while (len(unscraped)):
            # move unsraped_url to scraped_urls set
            url = unscraped.popleft()  # popleft(): Remove and return an element from the left side of the deque
            scraped.add(url)
            parts = urlsplit(url)    
            base_url = "{0.scheme}://{0.netloc}".format(parts)
            
            if '/' in parts.path:
                path = url[:url.rfind('/')+1]
            else:
                path = url
            print("Crawling URL %s" % url.encode(sys.stdout.encoding, errors='replace')) # Optional
            if not (url.endswith(".pdf") or url.endswith(".jpg") or url.endswith(".png")):
                response = get_source(url)
            
            if (response is None):
                continue
            
            # You may edit the regular expression as per your requirement
            new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+", 
                          response.text, re.I)) # re.I: (ignore case)
            new_emails = [email for email in new_emails if email.endswith('.com') or email.endswith('.ru') or email.endswith('.net')]
            print("Added more emails:", len(new_emails))
            
            # check netloc part of the url
            url_netloc = parts.netloc
            
            if url_netloc in visited_num_times:
                visited_num_times[url_netloc] = visited_num_times.get(url_netloc,0) + 1
            else:
                visited_num_times[url_netloc] = 1
            
            if url_netloc in visited_num_emails:
                visited_num_emails[url_netloc] = visited_num_emails.get(url_netloc,0) + len(new_emails)
            else:
                visited_num_emails[url_netloc] = len(new_emails)
                
            print("Current netloc: %s, visits: %d, netloc emails added: %d" %(url_netloc, visited_num_times[url_netloc], visited_num_emails[url_netloc]))
            if (visited_num_times[url_netloc] > 7) and (visited_num_emails[url_netloc] == 0):
                continue
            
            print("Len unscraped:", len(unscraped))
            emails.update(new_emails)
            emails_list.extend(emails)            
            
            # get more links
            if len_unscraped < max_links:
                print("Getting more links...")
                # create a beutiful soup for the html document
                soup = BeautifulSoup(response.text, 'lxml')
    
                for anchor in soup.find_all("a"): 
                    
                    # extract linked url from the anchor
                    if "href" in anchor.attrs:
                        link = anchor.attrs["href"]
                    else:
                        link = ''
                    
                    # resolve relative links (starting with /)
                    if link.startswith('/'):
                        link = base_url + link
                        
                    elif not link.startswith('http'):
                        link = path + link            
    
                    if not link.endswith(".gz"):
                        if not link in unscraped and not link in scraped:
                            unscraped.append(link)
                    # select max number of links
                    unscraped = collections.deque(itertools.islice(unscraped, 0, max_links))
                    
                    print(unscraped, len(unscraped))
                    len_unscraped = len(unscraped)
              
    #print(emails_list)
    return(emails_list)

In [7]:
# google search 
from googlesearch import search

# to search
query = "health practitioners iowa contact"

urls = []
for j in search(query, tld="co.in", num=10, start=5, stop=20, pause=2):
    urls.append(j)

urls = list(set(urls))
print(urls)

['https://health.iastate.edu/', 'https://www.polkcountyiowa.gov/health-department/clinical-services/nurse-practitioner-services/', 'https://www.mrhcia.com/', 'https://www.iowatotalcare.com/members/medicaid/find-a-doctor.html', 'https://www.greaterregional.org/', 'https://wholewomanhealthdsm.com/', 'https://www.wellmark.com/about/contact-us/iowa-providers', 'https://marionph.org/', 'https://www.iowafunctionalhealth.com/', 'https://iowacollegeaid.gov/HealthProfessionalRecruitmentProgram', 'https://www.siouxcenterhealth.org/contact/', 'https://www.iowaclinic.com/primary-care/specialties/internal-medicine/', 'http://www.211iowa.org/search/?id=31054845', 'https://studentlife.uiowa.edu/departments/student-health/', 'https://www.linncountyiowa.gov/592/Health-Services', 'https://www.public-health.uiowa.edu/', 'https://www.va.gov/iowa-city-health-care/work-with-us/jobs-and-careers/', 'https://iowapha.org/', 'https://www.adaircountyhealthsystem.org/family_practice_clinic.html', 'https://www.inte

In [8]:
emails_list = scrap_emails(urls, 10)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): health.iastate.edu:443


Crawling URL b'https://health.iastate.edu/'


C:\ApplicationsFolder\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'health.iastate.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
DEBUG:urllib3.connectionpool:https://health.iastate.edu:443 "GET / HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): health.iastate.edu:443


Added more emails: 0
Current netloc: health.iastate.edu, visits: 1, netloc emails added: 0
Len unscraped: 0
Getting more links...
deque(['https://health.iastate.edu']) 1
deque(['https://health.iastate.edu', 'https://health.iastate.edu/appointments/']) 2
deque(['https://health.iastate.edu', 'https://health.iastate.edu/appointments/', 'https://health.iastate.edu/clinic-hours/']) 3
deque(['https://health.iastate.edu', 'https://health.iastate.edu/appointments/', 'https://health.iastate.edu/clinic-hours/', 'https://health.iastate.edu/contact-us/']) 4
deque(['https://health.iastate.edu', 'https://health.iastate.edu/appointments/', 'https://health.iastate.edu/clinic-hours/', 'https://health.iastate.edu/contact-us/', 'https://health.iastate.edu/about-us/']) 5
deque(['https://health.iastate.edu', 'https://health.iastate.edu/appointments/', 'https://health.iastate.edu/clinic-hours/', 'https://health.iastate.edu/contact-us/', 'https://health.iastate.edu/about-us/', 'https://health.iastate.edu/cov

C:\ApplicationsFolder\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'health.iastate.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
DEBUG:urllib3.connectionpool:https://health.iastate.edu:443 "GET / HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): health.iastate.edu:443
C:\ApplicationsFolder\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'health.iastate.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Added more emails: 1
Current netloc: www.integrativemediowa.com, visits: 9, netloc emails added: 9
Len unscraped: 2
Crawling URL b'https://www.integrativemediowa.com/health-coaching.html'


C:\ApplicationsFolder\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.integrativemediowa.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
DEBUG:urllib3.connectionpool:https://www.integrativemediowa.com:443 "GET /health-coaching.html HTTP/1.1" 200 18494
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.integrativemediowa.com:443


Added more emails: 1
Current netloc: www.integrativemediowa.com, visits: 10, netloc emails added: 10
Len unscraped: 1
Crawling URL b'https://www.integrativemediowa.com/newborn.html'


C:\ApplicationsFolder\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.integrativemediowa.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
DEBUG:urllib3.connectionpool:https://www.integrativemediowa.com:443 "GET /newborn.html HTTP/1.1" 200 18148


Added more emails: 1
Current netloc: www.integrativemediowa.com, visits: 11, netloc emails added: 11
Len unscraped: 0


In [9]:
# filter out mail.ru - has anti-spam filtering
emails_list_f = [email for email in emails_list if not '@mail.ru' in email]
for substr in ['@example.com', '@email.com']:
    emails_list_f = [email for email in emails_list_f if not substr in email]
        
emails_set = list(set(emails_list_f))
print("Total emails collected:", len(emails_set))

Total emails collected: 16


In [10]:
for email in emails_set:
    print(email)

david.harden@recruiting.com
605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com
dd0a55ccb8124b9c9d938e3acf41f8aa@sentry.wixpress.com
test000@gmail.com
8eb368c655b84e029ed79ad7a5c1718e@sentry.wixpress.com
ConditionSupportReferral@wellmark.com
ConductEthicsQuestions@wellmark.com
wholewomanhealth@gmail.com
garrettm@wellmark.com
linn.block@mrhcia.com
831126cb46b74583bf6f72c5061cba9d@sentry-viewer.wixpress.com
1eeb89147c984dc6bc3ffafd9e6cd089@sentry.wixpress.com
blueink@wellmark.com
manning.regional@mrhcia.com
info@integrativemediowa.com
amy.benton@mrhcia.com
